# Работа с геоданными

In [713]:
!pip install folium

In [1]:
import folium
import pandas as pd
import branca.colormap as cm
import json

## Реализация вспомогательных методов

Для отрисовки сетки на карте потребуется описание сетки в формате json 

In [7]:
# Загрузка данных по регионам
region=pd.read_csv('regions.csv',sep=';')

# Фуникция возвращает форматированную пару координат:
# [40.748817, -73.985428]
def coord_to_str(longitude,latitude):
    return '['+str(longitude)+","+str(latitude)+']'

# Функция принимает ячейку region и возвращает 4 пары координат ячейки
def region_to_str(region):
    res=''
    delim=''
    for longitude,latitude in zip ([region.west,region.west,region.east,region.east],[region.south,region.north,region.north,region.south]):
        if len(res)>0: delim=','
        res=res+delim+coord_to_str(longitude,latitude)
    return res

# Функция возвращает описание ячейки в формате json 
def region_to_json(region,delim):
    id_=str(int(region.region))
    res=delim+'{\"type\":\"Feature\",\"id\":\"cell'+id_+'\",\"properties\":{\"name\":\"cell'+id_+'\"},\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[['+region_to_str(region)+']]}}'
    return res

# Формируем json файл с описанием сетки
def write_geo_json_data(filename,region):
    f=open(filename,'w')
    f.write('{\"type\":\"FeatureCollection\",\"features\":[')
    delim=''
    for r in region.iterrows():
        f.write(region_to_json(r[1],delim))
        delim=','
    f.write(']}')
    f.close()
    return filename
    
geo_json_data_filename=write_geo_json_data('region.json',region)
geo_json_data =json.load(open(geo_json_data_filename))

Расскраска ячеек сетки на карте будет проводиться на основании значений некой статистики на ячейках сетки.

Понадобится две статистики
1. **Суммарное** количество поездок из ячейки
2. **Среднее** количество поездок из ячейки

In [8]:
# Загружаем агрегированные данные за май
arg_data=pd.read_csv('arg_data.csv',sep=';')
arg_data.drop(labels='Unnamed: 0',axis=1,inplace=True)
# Строим статистику по среднему количеству поездок
mean_stat=arg_data.mean(axis=0)
mean_stat.index='cell'+mean_stat.index
# Строим статистику по сумме поездок
sum_stat=arg_data.sum(axis=0)
sum_stat.index='cell'+sum_stat.index

Реализуем функцию, которая  рисует карту города, накладывает на неё сетку и ставит метку.

Сетка накладывается исходя из json координат разметки, которые мы сформировали выше. Цвет ячеек сетки зависит от значения статистики функции в ячейке

In [9]:
# Функция возвращает постренную карту
def get_map(geo_json_data,stats,marker):
    # Функция строит карту цветов для отрисовки сетки поверх карты города
    # учитываем, что есть нулевые ячейки - белые
    # карта цветов 3х цветная - зеленый - мало поездок, оранжевый - средне, красный - много поездок
    def get_colormap(data):
        q_1=data[data>0].min()
        q_33=data[data>0].quantile(q=0.33)
        q_66=data[data>0].quantile(q=0.66)
        q_100=data.max()
        return cm.StepColormap(['white','green','orange','red'],index=[0,q_1*0.99,q_33,q_66,q_100],vmin=0,vmax=q_100)
    colormap=get_colormap(stats)
    # Рисуем карту города
    map_ = folium.Map(location=[40.7127, -74.0059],tiles='cartodbpositron',zoom_start=10)
    # Накладываем сетку
    folium.GeoJson(
        geo_json_data ,style_function=lambda feature: {
            'fillColor': colormap(stats[feature['id']]),
            'weight' : 0
            }).add_to(map_)
    # ставим метку
    folium.Marker(marker[0], popup=marker[1]).add_to(map_)
    return map_

## Выполнение заданий

### 1. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки

In [10]:
(sum_stat==0).sum()

1283

### Карта с сеткой по СУММЕ поездок и меткой Эмпайр-Стейт-Билдинг
$2.$ Нарисуйте статическую карту Нью-Йорка. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.

$3.$ Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.

In [11]:
# Строим карту и накладываем сетку по сумме поездок
marker_esb=[[40.748817, -73.985428],'Empire State Building']
map1=get_map(geo_json_data,sum_stat,marker_esb)
map1.save('map1.html')
map1

### Карта с сеткой по СРЕДНЕМУ ЧИСЛУ поездок и меткой Статуя свободы
$4.$ Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится статуя свободы.

$5.$ Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

In [12]:
# Строим карту и накладываем сетку по сумме поездок
marker_sl=[[40.6854, -74.0307],'Statue of Liberty']
map2=get_map(geo_json_data,mean_stat,marker_sl)
map2.save('map2.html')
map2

### 6. Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся. Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

In [19]:
mean_stat1=mean_stat
print ((mean_stat>=5).sum())
mean_stat1[mean_stat<5]=0


102


In [20]:
# Строим карту и накладываем сетку по сумме поездок
marker_sl=[[40.6854, -74.0307],'Statue of Liberty']
map3=get_map(geo_json_data,mean_stat1,marker_sl)
map3.save('map3.html')
map3